In [114]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import Adam, RMSprop, SGD
from torch.utils.data import DataLoader, Dataset

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [115]:
X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13, test_size=0.25)

In [116]:
class MyDataset(Dataset):
  def __init__(self, x,y):
    scale = StandardScaler()
    X_s = scale.fit_transform(x)
    x = torch.from_numpy(X_s.astype(np.float32))
    y = torch.from_numpy(y.astype(np.float32))
    self._base_datasets = x,y
  
  def __len__(self):
    return self._base_datasets[0].size(0)
  
  def __getitem__(self, idx):
      return tuple(base_dataset[idx] for base_dataset in self._base_datasets)

In [117]:
class MyNet(nn.Module):
  def __init__(self) -> None:
      super(MyNet, self).__init__()
      self.block_1 = nn.Sequential(
          nn.Linear(in_features=8, out_features=100, bias=True),
          nn.Dropout(0.1),
          nn.BatchNorm1d(100),
          nn.ReLU())
      self.block_2 = nn.Sequential(
          nn.Linear(in_features=100, out_features=100, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(100),
          nn.ReLU())
      self.block_3 = nn.Sequential(
          nn.Linear(in_features=100, out_features=60, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(60),
          nn.ReLU())
      self.block_4 = nn.Sequential(
          nn.Linear(in_features=60, out_features=30),
          nn.Dropout(0.2),
          nn.BatchNorm1d(30),
          nn.ReLU())
      self.predict = nn.Sequential(
          nn.Linear(in_features=30, out_features=1, bias=True),
          nn.BatchNorm1d(1),
          nn.ReLU())
  
  def forward(self, inp):
    out = self.block_1(inp)
    out = self.block_2(out)
    out = self.block_3(out)
    out = self.block_4(out)
    out = self.predict(out)
    return out[:, 0]

In [118]:
train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

In [119]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=0, drop_last=True)

In [120]:
net = MyNet()
optimizer = Adam(net.parameters(), lr=0.01)

In [121]:
def train_loop(train_loader, test_loader, net, optimizer):
  loss_fn = nn.MSELoss()
  best_acc = {'train': None, 'test': None}
  net.train()
  for epoch in range(10):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)

        # обнуляем градиент
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        
        # выводим статистику о процессе обучения
        if i % 150 == 0 or (i + 1) == len(train_loader):    # печатаем каждые 150 mini-batches
            net.eval()

            test_loss, test_running_total, test_loss  = 0.0, 0.0, 0.0
            for y, (out_test, lbl_test) in enumerate(test_loader):
                test_outputs = net(out_test)
                test_loss += loss_fn(test_outputs, lbl_test)
                test_running_total += len(lbl_test)
            
            res_loss_train = running_loss / running_items
            res_loss_test = test_loss / test_running_total
            
            if best_acc['train'] is None or res_loss_train < best_acc['train']:
              best_acc['train'] = res_loss_train
            
            if best_acc['test'] is None or res_loss_test < best_acc['test']:
              best_acc['test'] = res_loss_train

            print(f'Epoch [{epoch + 1}/{10}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {res_loss_train:.3f}. '\
                  f'Test acc: {res_loss_test:.3f}.')
            
            running_loss, running_items = 0.0, 0.0
            net.train()       
  print(f"Best acc train: {best_acc['train']:.3f}. Best acc test: {best_acc['test']:.3f}")
  print('Training is finished!')

In [122]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.059. Test acc: 0.088.
Epoch [1/10]. Step [151/241]. Loss: 0.026. Test acc: 0.017.
Epoch [1/10]. Step [241/241]. Loss: 0.011. Test acc: 0.015.
Epoch [2/10]. Step [1/241]. Loss: 0.008. Test acc: 0.015.
Epoch [2/10]. Step [151/241]. Loss: 0.009. Test acc: 0.028.
Epoch [2/10]. Step [241/241]. Loss: 0.008. Test acc: 0.054.
Epoch [3/10]. Step [1/241]. Loss: 0.009. Test acc: 0.054.
Epoch [3/10]. Step [151/241]. Loss: 0.008. Test acc: 0.044.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.025.
Epoch [4/10]. Step [1/241]. Loss: 0.007. Test acc: 0.025.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.012.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.032.
Epoch [5/10]. Step [1/241]. Loss: 0.005. Test acc: 0.033.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.023.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.014.
Epoch [6/10]. Step [1/241]. Loss: 0.008. Test acc: 0.015.
Epoch [6/10]. Step [151/241]. Loss: 0.007. Test acc:

In [124]:
net = MyNet()
optimizer = RMSprop(net.parameters(), lr=0.01)

In [125]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.074. Test acc: 0.088.
Epoch [1/10]. Step [151/241]. Loss: 0.015. Test acc: 0.028.
Epoch [1/10]. Step [241/241]. Loss: 0.009. Test acc: 0.015.
Epoch [2/10]. Step [1/241]. Loss: 0.008. Test acc: 0.015.
Epoch [2/10]. Step [151/241]. Loss: 0.008. Test acc: 0.049.
Epoch [2/10]. Step [241/241]. Loss: 0.007. Test acc: 0.033.
Epoch [3/10]. Step [1/241]. Loss: 0.009. Test acc: 0.024.
Epoch [3/10]. Step [151/241]. Loss: 0.008. Test acc: 0.012.
Epoch [3/10]. Step [241/241]. Loss: 0.007. Test acc: 0.044.
Epoch [4/10]. Step [1/241]. Loss: 0.006. Test acc: 0.049.
Epoch [4/10]. Step [151/241]. Loss: 0.007. Test acc: 0.012.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.030.
Epoch [5/10]. Step [1/241]. Loss: 0.005. Test acc: 0.027.
Epoch [5/10]. Step [151/241]. Loss: 0.007. Test acc: 0.053.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.009.
Epoch [6/10]. Step [1/241]. Loss: 0.005. Test acc: 0.008.
Epoch [6/10]. Step [151/241]. Loss: 0.006. Test acc:

In [126]:
net = MyNet()
optimizer = SGD(net.parameters(), lr=0.01)

In [127]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.051. Test acc: 0.082.
Epoch [1/10]. Step [151/241]. Loss: 0.024. Test acc: 0.011.
Epoch [1/10]. Step [241/241]. Loss: 0.012. Test acc: 0.009.
Epoch [2/10]. Step [1/241]. Loss: 0.011. Test acc: 0.009.
Epoch [2/10]. Step [151/241]. Loss: 0.010. Test acc: 0.010.
Epoch [2/10]. Step [241/241]. Loss: 0.010. Test acc: 0.008.
Epoch [3/10]. Step [1/241]. Loss: 0.010. Test acc: 0.008.
Epoch [3/10]. Step [151/241]. Loss: 0.009. Test acc: 0.008.
Epoch [3/10]. Step [241/241]. Loss: 0.009. Test acc: 0.008.
Epoch [4/10]. Step [1/241]. Loss: 0.008. Test acc: 0.008.
Epoch [4/10]. Step [151/241]. Loss: 0.008. Test acc: 0.008.
Epoch [4/10]. Step [241/241]. Loss: 0.007. Test acc: 0.008.
Epoch [5/10]. Step [1/241]. Loss: 0.009. Test acc: 0.008.
Epoch [5/10]. Step [151/241]. Loss: 0.008. Test acc: 0.018.
Epoch [5/10]. Step [241/241]. Loss: 0.007. Test acc: 0.009.
Epoch [6/10]. Step [1/241]. Loss: 0.005. Test acc: 0.010.
Epoch [6/10]. Step [151/241]. Loss: 0.007. Test acc:

In [ ]:
RMSprop проявил себя лучшим образом , как ни странно , хотя считается что Adam лучше , так как в нем учитывается и 
идея накопления движения и идея более слабого обновления весов для типичных признаков.